# ✅ Demanda de Consolidação de Base de Dados

## 🧠 **Objetivo:** Gerar Conjunto de Dados Consolidado Final

### 1. Dados de Cicatrizes de Queimadas 🔥
### 2. Dados Nº de Focus de Incêndio, Nº de Internações e Emissão de PM 2.5 🔥

Será escrita uma sintaxe em Python, fazendo uso das seguintes bibliotecas:

* `os`
* `glob`
* `numpy`
* `pandas`

Tendo como finalidade gerar conjunto de dados que contenha informações como **Nº de Hospitalizações**; **Nº de Focos de Incêndio** e; a **Emissão de PM 2.5** *Média*, *Mediana* e *Máxima* de **2005** a **2024** por *Município*, *Ano* e *Mês*.

---

* **Requerente:** CEL QOBM WAGNER
* **Analista:** VC Cauã Rodrigues

---

Para o Google Colaboratory:
```python
# Conexão com o Google Drive
from google.colab import drive
drive.mount("/content/drive")
```

# (1) Dados de Cicatrizes de Queimadas

```python
!pip install rioxarray

import os
import glob
import xarray as xr
import rioxarray
import pandas as pd
import numpy as np

# Definir o caminho da pasta onde estão os arquivos
path_to_files = "/content/drive/MyDrive/CBMPA/DEMANDAS INDIVIDUAIS/DADOS DE QUEIMADAS/DADOS TIF/"
file_pattern = "mapbiomas_fire_col4_br_annual_burned_*.tif"

# Listar e ordenar os arquivos para garantir a sequência cronológica
files = sorted(glob.glob(os.path.join(path_to_files, file_pattern)))

# Extrair os anos dos nomes dos arquivos
years = [int(f.split('_')[-1].split('.')[0]) for f in files]

# Função de leitura com CHUNKS (ativando o Dask)
def load_with_dask(file, year):
    # Definimos chunks de 1000x1000 pixels para não sobrecarregar a RAM
    da = rioxarray.open_rasterio(file, chunks={'x': 1000, 'y': 1000})
    da = da.expand_dims(time=[pd.to_datetime(year, format='%Y')])
    return da

# Concatenar (isso será instantâneo, pois ele não lerá os dados de fato agora)
data_cube = xr.concat([load_with_dask(f, y) for f, y in zip(files, years)], dim="time")

# Limpeza de dimensões desnecessárias
if 'band' in data_cube.dims:
    data_cube = data_cube.squeeze("band").drop_vars("band")

print("Cubo de dados mapeado com sucesso (Lazy Loading ativo)!")

data_cube
```

# (2) Dados Nº de Focus de Incêndio, Nº de Internações e Emissão de PM 2.5

In [1]:
import os
import glob
import pandas as pd

In [ ]:
# Mapeamento dos meses para categorização
month_mapping = {
    1: "Janeiro", 2: "Fevereiro", 3: "Março", 4: "Abril", 5: "Maio", 6: "Junho",
    7: "Julho", 8: "Agosto", 9: "Setembro", 10: "Outubro", 11: "Novembro", 12: "Dezembro"
}

## (2.1) Dados de Focos de Incêndio - 1995 à 2024

### (A) Leitura Automatizda

In [2]:
# Definir o caminho da pasta onde estão os arquivos
path_to_files = "C:/Users/breno/Documents/CBMPA/DEMANDAS INDIVIDUAIS/CEL WAGNER/001 - Arquivos Excel Focos 1995-2024/"

# Listar os arquivos .CSV
list_of_files = glob.glob(os.path.join(path_to_files, "*.csv"))

# Ordenar
list_of_files.sort()

# Leitura automatizada dos dados
list_of_dfs = [
    pd.read_csv(
        file_path, 
        dtype={"Municipio": "string"}, 
        parse_dates=["DataHora"]
    ) for file_path in list_of_files
]

# Concatenação dos dados
df_concat = pd.concat(list_of_dfs, ignore_index=True)

# Info do Data Frame
display(df_concat.info())

<class 'pandas.DataFrame'>
RangeIndex: 1116628 entries, 0 to 1116627
Data columns (total 12 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   DataHora      1116628 non-null  object
 1   Satelite      1116628 non-null  object
 2   Pais          1116628 non-null  object
 3   Estado        1116628 non-null  object
 4   Municipio     1116628 non-null  string
 5   Bioma         1116628 non-null  object
 6   DiaSemChuva   411114 non-null   object
 7   Precipitacao  411114 non-null   object
 8   RiscoFogo     411114 non-null   object
 9   Latitude      1116628 non-null  object
 10  Longitude     1116628 non-null  object
 11  FRP           1026670 non-null  object
dtypes: object(11), string(1)
memory usage: 102.2+ MB


None

In [3]:
# Exibir as primeiras linhas do DataFrame
display(df_concat.head())
print("\n")
# Exibir as últimas linhas do DataFrame
display(df_concat.tail())

,DataHora,Satelite,Pais,Estado,Municipio,Bioma,DiaSemChuva,Precipitacao,RiscoFogo,Latitude,Longitude,FRP
0,1998-07-23 21:49:19,NOAA-12,Brasil,PARÁ,ALTAMIRA,Amazônia,NaN,NaN,NaN,-6.93333,-55.17167,NaN
1,1998-08-06 21:40:47,NOAA-12,Brasil,PARÁ,BANNACH,Amazônia,NaN,NaN,NaN,-7.54,-50.40167,NaN
2,1998-08-08 20:55:37,NOAA-12,Brasil,PARÁ,BANNACH,Amazônia,NaN,NaN,NaN,-7.52667,-50.72167,NaN
3,1998-08-08 20:55:37,NOAA-12,Brasil,PARÁ,XINGUARA,Amazônia,NaN,NaN,NaN,-6.85667,-49.295,NaN
4,1998-08-12 21:09:00,NOAA-12,Brasil,PARÁ,PIÇARRA,Amazônia,NaN,NaN,NaN,-6.42333,-49.335,NaN


,DataHora,Satelite,Pais,Estado,Municipio,Bioma,DiaSemChuva,Precipitacao,RiscoFogo,Latitude,Longitude,FRP
1116623,2024-12-23 17:32:00,AQUA_M-T,Brasil,PARÁ,PACAJÁ,Amazônia,0,10.38,0.0,-3.4902,-50.23558,67.3
1116624,2024-12-23 17:30:00,AQUA_M-T,Brasil,PARÁ,ULIANÓPOLIS,Amazônia,3,0.38,0.1,-3.70561,-47.61203,7.5
1116625,2024-12-23 17:32:00,AQUA_M-T,Brasil,PARÁ,TOMÉ-AÇU,Amazônia,0,0.29,0.01,-2.30686,-48.33688,13.4
1116626,2024-12-23 17:32:00,AQUA_M-T,Brasil,PARÁ,AURORA DO PARÁ,Amazônia,0,0.0,0.16,-2.28926,-47.95486,10.7
1116627,2024-12-23 17:32:00,AQUA_M-T,Brasil,PARÁ,MEDICILÂNDIA,Amazônia,3,0.0,0.24,-3.0815,-52.92336,22.3


* **Nota:** os dados foram checados manualmente antes de virem para o Python. Por isso foi utilizada essa abordagem de leitura automatiada.

### (B) Tratamento de Dados

In [ ]:
# Convertendo a coluna "DataHora" para o formato datetime
df_concat["DATA"] = pd.to_datetime(df_concat["DataHora"], errors="coerce")

# Extraindo a hora
df_concat["HORA"] = df_concat["DATA"].dt.time

# Excluir o atributo "DataHora"
df_concat.drop(columns=["DataHora"], inplace=True)

# Extrair o atributo "ANO" da atributo "DATA"
df_concat["ANO"] = df_concat["DATA"].dt.year

# Executar filtro para selecionar os dados a partir do ano de 2005
df_concat = df_concat[df_concat["ANO"] >= 2005]

# Extraindo o atributo "MÊS" da atributo "DATA"
df_concat["MÊS"] = df_concat["DATA"].dt.month

# Extraindo o atributo "DIA" da atributo "DATA"
df_concat["DIA"] = df_concat["DATA"].dt.day

# Aplicando o mapeamento para categorizar os meses (equivalente a Factor em R)
df_concat["MÊS"] = pd.Categorical(
    df_concat["MÊS"].map(month_mapping),
    categories=list(month_mapping.values()),
    ordered=True
)

# Ordenar Data Frame por "DATA"
df_concat.sort_values(by="DATA", inplace=True)

# Ajuste do nome dos atrubutos para caixa alta
df_concat.columns = df_concat.columns.str.upper()

# Resetar o índice
df_concat.reset_index(drop=True, inplace=True)

# Exibir informações do DataFrame após as transformações
display(df_concat.info())

<class 'pandas.DataFrame'>
RangeIndex: 842061 entries, 0 to 842060
Data columns (total 16 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   SATELITE      842061 non-null  object        
 1   PAIS          842061 non-null  object        
 2   ESTADO        842061 non-null  object        
 3   MUNICIPIO     842061 non-null  string        
 4   BIOMA         842061 non-null  object        
 5   DIASEMCHUVA   411114 non-null  object        
 6   PRECIPITACAO  411114 non-null  object        
 7   RISCOFOGO     411114 non-null  object        
 8   LATITUDE      842061 non-null  object        
 9   LONGITUDE     842061 non-null  object        
 10  FRP           840046 non-null  object        
 11  DATA          842061 non-null  datetime64[us]
 12  HORA          842061 non-null  object        
 13  ANO           842061 non-null  int32         
 14  MÊS           842061 non-null  category      
 15  DIA           842061 non-nul

None

In [5]:
# Exibir as primeiras linhas do DataFrame após as transformações
display(df_concat.head())
print("\n")
# Exibir as últimas linhas do DataFrame após as transformações
display(df_concat.tail())

,SATELITE,PAIS,ESTADO,MUNICIPIO,BIOMA,DIASEMCHUVA,PRECIPITACAO,RISCOFOGO,LATITUDE,LONGITUDE,FRP,DATA,HORA,ANO,MÊS,DIA
0,AQUA_M-T,Brasil,PARÁ,ÓBIDOS,Amazônia,NaN,NaN,NaN,-1.926,-55.315,18.6,2005-01-01 17:28:00,17:28:00,2005,Janeiro,1
1,AQUA_M-T,Brasil,PARÁ,ALENQUER,Amazônia,NaN,NaN,NaN,-1.91,-54.503,11.8,2005-01-01 17:28:00,17:28:00,2005,Janeiro,1
2,AQUA_M-T,Brasil,PARÁ,MONTE ALEGRE,Amazônia,NaN,NaN,NaN,-1.354,-54.07,13.6,2005-01-01 17:28:00,17:28:00,2005,Janeiro,1
3,AQUA_M-T,Brasil,PARÁ,MONTE ALEGRE,Amazônia,NaN,NaN,NaN,-1.566,-54.126,21.3,2005-01-01 17:28:00,17:28:00,2005,Janeiro,1
4,AQUA_M-T,Brasil,PARÁ,MOJUÍ DOS CAMPOS,Amazônia,NaN,NaN,NaN,-2.71,-54.727,85.8,2005-01-01 17:28:00,17:28:00,2005,Janeiro,1


,SATELITE,PAIS,ESTADO,MUNICIPIO,BIOMA,DIASEMCHUVA,PRECIPITACAO,RISCOFOGO,LATITUDE,LONGITUDE,FRP,DATA,HORA,ANO,MÊS,DIA
842056,AQUA_M-T,Brasil,PARÁ,MONTE ALEGRE,Amazônia,1,1.1,0.03,-1.81534,-53.97981,11.8,2024-12-31 17:53:00,17:53:00,2024,Dezembro,31
842057,AQUA_M-T,Brasil,PARÁ,ALMEIRIM,Amazônia,1,8.62,0.01,-1.21053,-52.02901,20.8,2024-12-31 17:53:00,17:53:00,2024,Dezembro,31
842058,AQUA_M-T,Brasil,PARÁ,ALMEIRIM,Amazônia,1,8.82,0.01,-1.21808,-52.0189,16.1,2024-12-31 17:53:00,17:53:00,2024,Dezembro,31
842059,AQUA_M-T,Brasil,PARÁ,CURRALINHO,Amazônia,0,6.82,0.0,-1.6198,-49.83651,5.6,2024-12-31 17:53:00,17:53:00,2024,Dezembro,31
842060,AQUA_M-T,Brasil,PARÁ,SANTARÉM,Amazônia,1,6.43,0.12,-2.27447,-54.47422,10.4,2024-12-31 17:53:00,17:53:00,2024,Dezembro,31


### (C) Salvar Dados após Pré processamento

In [6]:
# Salvar o Conjunto de Dados em Excel
df_concat.to_excel("C:/Users/breno/Documents/CBMPA/DEMANDAS INDIVIDUAIS/CEL WAGNER/data_focus_fire_2005_2024.xlsx", index=False)

# Salvar o Conjunto de Dados em CSV
df_concat.to_csv("C:/Users/breno/Documents/CBMPA/DEMANDAS INDIVIDUAIS/CEL WAGNER/data_focus_fire_2005_2024.csv", index=False)

## (2.2) Agrupamento com os Dados de:
* ### Nº de Internações Hospitalares
* ### Emissão de *PM 2.5* (https://zenodo.org/records/16374139?utm_source=chatgpt.com)

In [ ]:
from geobr import read_municipality
import re
import unicodedata

In [8]:
# Gerar um Data Frame simplificado p/ agrupamento
df_simplified = pd.merge(
    left=df_concat.groupby(by=["MUNICIPIO", "ANO", "MÊS"], as_index=False)["DIA"].count().rename(columns={"DIA": "CONTAGEM_FOCOS"}),
    right=df_concat.groupby(by=["MUNICIPIO", "ANO", "MÊS"], as_index=False)["DIASEMCHUVA"].sum().rename(columns={"DIASEMCHUVA": "CONTAGEM_DIAS_SEM_CHUVA"}),
    on=["MUNICIPIO", "ANO", "MÊS"], # Correspondência dos atributos para o merge
    how="outer"                     # Método de junção (inner, left, right, outer)
)

# Para facilitar a junção com os demais dados
df_simplified.columns = df_simplified.columns.str.lower()

### (A) Tratamento para os Dados Hospitalares

In [9]:
# Definir o caminho da pasta dos arquivos de dados de Hospitalizações
path_to_files_of_hosp = "C:/Users/breno/Documents/CBMPA/DEMANDAS INDIVIDUAIS/CEL WAGNER/DADOS DE HOPITALIZAÇÕES - 1998 A 2024/"

# Listar os arquivos .XLSX
list_of_files_of_hosp = glob.glob(os.path.join(path_to_files_of_hosp, "*.xlsx"))

# Ordenar
list_of_files_of_hosp.sort()

# Leitura dos dados
dfHosp_98_07, dfHosp_08_24 = pd.read_excel(list_of_files_of_hosp[0]), pd.read_excel(list_of_files_of_hosp[1])

# Informações dos dados de 1998 a 2007
display(dfHosp_98_07.info())

# Informações dos dados de 2008 a 2024
display(dfHosp_08_24.info())

<class 'pandas.DataFrame'>
RangeIndex: 143 entries, 0 to 142
Columns: 121 entries, Município to 2007-12-01 00:00:00
dtypes: int64(1), object(119), str(1)
memory usage: 135.3+ KB


None

<class 'pandas.DataFrame'>
RangeIndex: 144 entries, 0 to 143
Columns: 211 entries, Município to 2024-12-01 00:00:00
dtypes: int64(1), object(209), str(1)
memory usage: 237.5+ KB


None

In [ ]:
# Função de Pivotagem ajustada para os dados de hospitalizações
def pivot_hospitalizations(df, id_vars):
    """
    Função (não genérica) para pivotar os dados de hospitalizações, transformando as colunas de anos em uma única coluna de datas e os valores correspondentes em uma coluna de contagem de hospitalizações.
    Parâmetros:
    - df: DataFrame contendo os dados de hospitalizações.
    - id_vars: Lista de colunas que devem permanecer como identificadores (ex: "Município").
    Retorna:
    - df_pivoted: DataFrame pivotado com as colunas de anos transformadas em uma coluna de datas e os valores correspondentes em uma coluna de contagem de hospitalizações.
    """
    # Atributos que serão pivotados
    value_vars = df.columns.difference(id_vars).tolist()

    # Verificar se os atributos existem no DataFrame
    if not all(var in df.columns for var in id_vars + value_vars):
        raise ValueError("Alguns dos atributos especificados não existem no DataFrame.")
    
    # Realizar a pivotagem
    df_pivoted = df.melt(
        id_vars=id_vars,
        value_vars=value_vars,
        var_name="date",
        value_name="n_hospitalizations"
    )

    # Ajuste no atributo "Município"
    df_pivoted["Município"] = df_pivoted["Município"].str.split().str[1:].str.join(" ")

    # Ajuste no atributo "n_hospitalizations" para garantir que seja numérico
    df_pivoted["n_hospitalizations"] = pd.to_numeric(
        arg=df_pivoted["n_hospitalizations"],
        errors="coerce"
    ).fillna(0).astype(int)

    # Coverter o atributo "date" para o formato datetime
    df_pivoted["date"] = pd.to_datetime(
        arg=df_pivoted["date"],
        errors="coerce"
    )

    # Extração dos atributos "ano", "mês" e "trimestre" a partir do atributo "date"
    df_pivoted["ano"] = df_pivoted["date"].dt.year
    df_pivoted["mês"] = df_pivoted["date"].dt.month
    df_pivoted["trimestre"] = df_pivoted["date"].dt.quarter

    # Aplicando o mapeamento para categorizar os meses (equivalente a Factor em R)
    df_pivoted["mês"] = pd.Categorical(
        df_pivoted["mês"].map(month_mapping),
        categories=list(month_mapping.values()),
        ordered=True
    )

    # Criar atributo "período" (Chuvoso/Não Chuvoso) com base no atributo "mês"
    df_pivoted["período"] = df_pivoted["mês"].apply(
        lambda x: "Chuvoso" if x in ["Dezembro", "Janeiro", "Fevereiro", "Março", "Abril", "Maio"] else "Não Chuvoso"
    )

    # Retornar o DataFrame pivotado
    return df_pivoted

In [11]:
# Gerer um Data Frama concatenado
df_concat_hosp = pd.concat(
    objs=[
    pivot_hospitalizations(dfHosp_98_07, id_vars=["Município"]),
    pivot_hospitalizations(dfHosp_08_24, id_vars=["Município"])
    ],
    ignore_index=True
)

In [12]:
# Salvar o Conjunto de Dados em Excel
df_concat_hosp.to_excel("C:/Users/breno/Documents/CBMPA/DEMANDAS INDIVIDUAIS/CEL WAGNER/data_hospitalizations_1998_2024.xlsx", index=False)

# Salvar o Conjunto de Dados em CSV
df_concat_hosp.to_csv("C:/Users/breno/Documents/CBMPA/DEMANDAS INDIVIDUAIS/CEL WAGNER/data_hospitalizations_1998_2024.csv", index=False)

### (B) Tratamento para os Dados de Emissão de *PM 2.5*

In [13]:
# Definir caminho para os dados de PM 2.5
path_to_pm25 = "C:\\Users\\breno\\Documents\\CBMPA\\DEMANDAS INDIVIDUAIS\\CEL WAGNER\\GEO\\pm25_max_mean.csv"

# Leitura dos dados de PM 2.5
df_pm25 = pd.read_csv(path_to_pm25, parse_dates=["date"])

# Informações do DataFrame de PM 2.5
display(df_pm25.info())

<class 'pandas.DataFrame'>
RangeIndex: 44720340 entries, 0 to 44720339
Data columns (total 3 columns):
 #   Column     Dtype         
---  ------     -----         
 0   code_muni  float64       
 1   date       datetime64[us]
 2   value      float64       
dtypes: datetime64[us](1), float64(2)
memory usage: 1023.6 MB


None

In [14]:
# Exibir as primeiras linhas do DataFrame
display(df_pm25.head())
print("\n")
# Exibir as últimas linhas do DataFrame
display(df_pm25.tail())

,code_muni,date,value
0,1100015.0,2003-01-01,14.02
1,1100023.0,2003-01-01,23.56
2,1100031.0,2003-01-01,15.46
3,1100049.0,2003-01-01,14.76
4,1100056.0,2003-01-01,14.20


,code_muni,date,value
44720335,5222005.0,2024-12-31,19.59
44720336,5222054.0,2024-12-31,13.07
44720337,5222203.0,2024-12-31,17.59
44720338,5222302.0,2024-12-31,18.23
44720339,5300108.0,2024-12-31,26.18


In [15]:
# Executar filtro para selecionar apenas dados do Estado do Pará
df_pm25 = df_pm25[df_pm25["code_muni"].astype(str).str.startswith("15")]

# Ajuste no atributo "code_muni" para manter apenas os 6 dígitos
df_pm25["code_muni"] = df_pm25["code_muni"].astype(str).str[:7].astype(int)

# Resetar o índice do DataFrame de PM 2.5
df_pm25.reset_index(drop=True, inplace=True)

# Informações do DataFrame de PM 2.5 após o filtro
display(df_pm25.info())

<class 'pandas.DataFrame'>
RangeIndex: 1149148 entries, 0 to 1149147
Data columns (total 3 columns):
 #   Column     Non-Null Count    Dtype         
---  ------     --------------    -----         
 0   code_muni  1149148 non-null  int64         
 1   date       1149148 non-null  datetime64[us]
 2   value      1149148 non-null  float64       
dtypes: datetime64[us](1), float64(1), int64(1)
memory usage: 26.3 MB


None

In [16]:
# Exibir as primeiras linhas do DataFrame
display(df_pm25.head())
print("\n")
# Exibir as últimas linhas do DataFrame
display(df_pm25.tail())

,code_muni,date,value
0,1500107,2003-01-01,16.62
1,1500131,2003-01-01,18.21
2,1500206,2003-01-01,30.71
3,1500305,2003-01-01,15.82
4,1500347,2003-01-01,10.54


,code_muni,date,value
1149143,1508159,2024-12-31,12.27
1149144,1508209,2024-12-31,17.14
1149145,1508308,2024-12-31,19.76
1149146,1508357,2024-12-31,20.31
1149147,1508407,2024-12-31,12.66


In [ ]:
# Leitura dos dados dos municípios utilizando a biblioteca geobr
df_muni = read_municipality(code_muni=15, year=2024)[["code_muni", "name_muni"]]

# Ajuste do tipo de dado do atributo "code_muni" para inteiro
df_muni["code_muni"] = df_muni["code_muni"].astype(int)

# Integrar os dados de PM 2.5 com os nomes dos municípios
df_pm25 = pd.merge(
    left=df_pm25,
    right=df_muni,
    on="code_muni",
    how="left"
)

# Criar atributo de ano e mês a partir do atributo "date"
df_pm25["ano"] = df_pm25["date"].dt.year
df_pm25["mês"] = df_pm25["date"].dt.month

# Executar filtro para selecionar os dados a partir do ano de 2005
df_pm25 = df_pm25[df_pm25["ano"] >= 2005]

# Aplicando o mapeamento para categorizar os meses (equivalente a Factor em R)
df_pm25["mês"] = pd.Categorical(
    df_pm25["mês"].map(month_mapping),
    categories=list(month_mapping.values()),
    ordered=True
)

# Ordenar Data Frame por "date"
df_pm25.sort_values(by="date", inplace=True)

# Informações do Data Frame integrado
display(df_pm25.info())

<class 'pandas.DataFrame'>
Index: 1044615 entries, 104533 to 1149147
Data columns (total 6 columns):
 #   Column     Non-Null Count    Dtype         
---  ------     --------------    -----         
 0   code_muni  1044615 non-null  int64         
 1   date       1044615 non-null  datetime64[us]
 2   value      1044615 non-null  float64       
 3   name_muni  1044615 non-null  str           
 4   ano        1044615 non-null  int32         
 5   mês        1044615 non-null  category      
dtypes: category(1), datetime64[us](1), float64(1), int32(1), int64(1), str(1)
memory usage: 44.8 MB


None

In [18]:
# Agrupar dados por município, ano e mês, calculando mpetricas para PM 2.5 (atributo "value")
df_pm25_grouped = df_pm25.groupby(
    by=["code_muni", "name_muni", "ano", "mês"], 
    as_index=False
).agg(
    {
        "value": ["mean", "median", "max"],
        "ano": "first",
        "mês": "first"
    }
)

# Renomear os atributos agregados de PM 2.5 para facilitar a interpretação
df_pm25_grouped.columns = [
    col[0] if col[1] in ["", "first"] else "_".join(col).strip() for col in df_pm25_grouped.columns
]

In [19]:
# Exibir as primeiras linhas do DataFrame
display(df_pm25_grouped.head())
print("\n")
# Exibir as últimas linhas do DataFrame
display(df_pm25_grouped.tail())

,code_muni,name_muni,value_mean,value_median,value_max,ano,mês
0,1500107,Abaetetuba,24.933548,21.650,58.29,2005,Janeiro
1,1500107,Abaetetuba,17.760000,16.870,35.87,2005,Fevereiro
2,1500107,Abaetetuba,23.149032,23.370,56.01,2005,Março
3,1500107,Abaetetuba,21.093000,20.475,39.13,2005,Abril
4,1500107,Abaetetuba,16.384839,15.680,26.84,2005,Maio


,code_muni,name_muni,value_mean,value_median,value_max,ano,mês
34315,1508407,Xinguara,53.415806,43.07,181.79,2024,Agosto
34316,1508407,Xinguara,140.880333,92.89,693.63,2024,Setembro
34317,1508407,Xinguara,39.130000,36.77,86.83,2024,Outubro
34318,1508407,Xinguara,46.481667,45.34,75.17,2024,Novembro
34319,1508407,Xinguara,28.686452,27.98,49.71,2024,Dezembro


In [20]:
# Salvar o Conjunto de Dados em Excel
df_pm25_grouped.to_excel("C:/Users/breno/Documents/CBMPA/DEMANDAS INDIVIDUAIS/CEL WAGNER/data_pm25_2005_2024.xlsx", index=False)

# Salvar o Conjunto de Dados em CSV
df_pm25_grouped.to_csv("C:/Users/breno/Documents/CBMPA/DEMANDAS INDIVIDUAIS/CEL WAGNER/data_pm25_2005_2024.csv", index=False)

### (C) Junção dos Dados

In [21]:
def str_squish_python(text):
    # Remove espaços em branco do início e fim, depois substitui múltiplos espaços por um único
    return re.sub(r'\s+', ' ', text).strip()

def remove_accents_python(text):
    nfkd_form = unicodedata.normalize('NFKD', text)
    return ''.join([c for c in nfkd_form if not unicodedata.combining(c)])

In [22]:
# Ajuste para garantir a correspondência dos nomes dos municípios entre os DataFrames: de focos de incêndio e hospitalizações
df_simplified["name_muni"] = df_simplified["municipio"].apply(
    lambda x: remove_accents_python(str_squish_python(x)).upper() if pd.notnull(x) else x
)
# de hospitalizações
df_concat_hosp["name_muni"] = df_concat_hosp["Município"].apply(
    lambda x: remove_accents_python(str_squish_python(x)).upper() if pd.notnull(x) else x
)
# e de PM 2.5
df_pm25_grouped["name_muni"] = df_pm25_grouped["name_muni"].apply(
    lambda x: remove_accents_python(str_squish_python(x)).upper() if pd.notnull(x) else x
)

# Excluir atributo "MUNICIPIO" do DataFrame de focos de incêndio e o atributo "Município" do DataFrame de hospitalizações para evitar redundância após a criação do atributo "name_muni"
df_simplified.drop(columns=["municipio"], inplace=True)
df_concat_hosp.drop(columns=["Município", "date"], inplace=True)

In [23]:
# Executar junção entre os DataFrames de focos de incêndio e hospitalizações com base no atributo "name_muni"
df_joined = pd.merge(
    left=df_simplified,
    right=df_concat_hosp,
    on=["name_muni", "ano", "mês"],
    how="left"
).merge(
    right=df_pm25_grouped,
    on=["name_muni", "ano", "mês"],
    how="left"
)

# Visualizar o Data Frame resultante da junção
display(df_joined)

,ano,mês,contagem_focos,contagem_dias_sem_chuva,name_muni,n_hospitalizations,trimestre,período,code_muni,value_mean,value_median,value_max
0,2005,Janeiro,11,0,ABAETETUBA,121.0,1.0,Chuvoso,1500107.0,24.933548,21.650,58.29
1,2005,Junho,2,0,ABAETETUBA,141.0,2.0,Não Chuvoso,1500107.0,12.477667,10.650,25.77
2,2005,Julho,5,0,ABAETETUBA,156.0,3.0,Não Chuvoso,1500107.0,12.701935,10.320,24.48
3,2005,Agosto,16,0,ABAETETUBA,126.0,3.0,Não Chuvoso,1500107.0,14.380323,13.110,36.10
4,2005,Setembro,11,0,ABAETETUBA,123.0,3.0,Não Chuvoso,1500107.0,22.648667,20.480,44.93
...,...,...,...,...,...,...,...,...,...,...,...,...
20142,2024,Agosto,69,458,OBIDOS,14.0,3.0,Não Chuvoso,1505106.0,14.984194,12.220,31.79
20143,2024,Setembro,94,246,OBIDOS,12.0,3.0,Não Chuvoso,1505106.0,19.674333,17.770,50.24
20144,2024,Outubro,156,516,OBIDOS,27.0,4.0,Não Chuvoso,1505106.0,22.537097,18.120,59.57
20145,2024,Novembro,243,951,OBIDOS,23.0,4.0,Não Chuvoso,1505106.0,34.455000,31.475,107.79


In [24]:
# Salvar o Conjunto de Dados em Excel
df_joined.to_excel("C:/Users/breno/Documents/CBMPA/DEMANDAS INDIVIDUAIS/CEL WAGNER/data_complete.xlsx", index=False)

# Salvar o Conjunto de Dados em CSV
df_joined.to_csv("C:/Users/breno/Documents/CBMPA/DEMANDAS INDIVIDUAIS/CEL WAGNER/data_complete.csv", index=False)